In [1]:
import pandas as pd

og_df = pd.read_pickle('df.pkl') 
count_df1 = og_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='read_count')
count_df2 = og_df.groupby(['transcript_id']).size().reset_index(name='expression_count') #most genes are likely to produce unique transcripts
df = og_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
df = df.merge(count_df2, on=['transcript_id'])

agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'sum'
}

summary_df = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()

summary_df = summary_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
summary_df = summary_df.merge(count_df2, on=['transcript_id'])

summary_df['label_percentage'] = summary_df['label'] / summary_df['read_count'] * 100
summary_df = summary_df.rename(columns={'label': 'label_sum'})
summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

In [2]:
agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'first'
}

summary_df1 = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()
summary_df1['label'].equals(summary_df['label'])

True

In [3]:
summary_df['label_percentage'].unique()

array([  0., 100.])

In [4]:
summary_df = summary_df.drop(columns=['label_percentage', 'label_sum'])
summary_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count,label
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,185,3449,0
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,172,3449,0
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,185,3449,0
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,200,3449,0
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,198,3449,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,73,1323,1
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,69,1323,0
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,84.190625,64,1323,1
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,57,1323,0


In [5]:
print(df['label'].value_counts())
print(summary_df['label'].value_counts())

label
0    10527532
1      499574
Name: count, dtype: int64
label
0    116363
1      5475
Name: count, dtype: int64


### 5-mer feature and binary features for first and last letters

In [6]:
# function to convert features
def sequence_to_features(seq):
    """
    Convert a 7-letter sequence into binary features for the first and last letters.
    
    Parameters:
    - seq: A string of 7 letters (A, T, C, G)
    
    Returns:
    - A dictionary with binary features for the first and last letters.
    """
    letters = ['A', 'T', 'C', 'G']
    features = {}
    
    first_char = seq[0]
    last_char = seq[-1]
    
    for letter in letters:
        features[f"1{letter}"] = first_char == letter
        features[f"7{letter}"] = last_char == letter
            
    return features

In [7]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [8]:
new_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,expression_count,label,1A,7A,1T,7T,1C,7C,1G,7G
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,3449,0,True,True,False,False,False,False,False,False
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,3449,0,False,False,False,False,True,False,False,True
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,3449,0,False,False,False,False,False,False,True,True
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,3449,0,True,False,False,True,False,False,False,False
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,3449,0,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1323,1,False,False,False,True,False,False,True,False
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,1323,0,False,False,False,False,True,True,False,False
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1323,1,False,False,True,False,False,True,False,False
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,1323,0,False,True,False,False,True,False,False,False


In [9]:
new_df['5-mer'] = new_df['combined nucleotides'].apply(lambda x: x[1:-1])
new_df.groupby(['5-mer'])['label'].sum().sort_values(ascending=False)

5-mer
GGACT    1440
GAACT     783
GGACA     691
GGACC     552
AGACT     525
TGACT     430
AAACT     255
GAACA     187
AGACA     147
GAACC     124
TGACC      90
TAACT      71
TGACA      68
AGACC      60
AAACA      38
AAACC      11
TAACC       2
TAACA       1
Name: label, dtype: int64

In [10]:
new_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,label,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,0,True,True,False,False,False,False,False,False,AGACC
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,0,False,False,False,False,True,False,False,True,AAACT
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,0,False,False,False,False,False,False,True,True,AAACA
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,0,True,False,False,True,False,False,False,False,GAACA
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,0,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1,False,False,False,True,False,False,True,False,GGACA
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,0,False,False,False,False,True,True,False,False,TGACA
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1,False,False,True,False,False,True,False,False,GGACA
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,0,False,True,False,False,True,False,False,False,TGACC


### Label Encoder

In [11]:
from sklearn.preprocessing import LabelEncoder

combined = new_df.copy()

label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined['gene_id'] = label_encoder.fit_transform(combined['gene_id'])
combined

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,label,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,0,244,10,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,0,True,True,False,False,False,False,False,False,AGACC
1,0,261,10,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,0,False,False,False,False,True,False,False,True,AAACT
2,0,316,10,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,0,False,False,False,False,False,False,True,True,AAACA
3,0,332,10,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,0,True,False,False,True,False,False,False,False,GAACA
4,0,368,10,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,0,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1,False,False,False,True,False,False,True,False,GGACA
121834,5332,1429,2764,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,0,False,False,False,False,True,True,False,False,TGACA
121835,5332,1531,2764,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1,False,False,True,False,False,True,False,False,GGACA
121836,5332,1537,2764,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,0,False,True,False,False,True,False,False,False,TGACC


In [12]:
combined = pd.get_dummies(combined, columns=['5-mer'], drop_first=False)
combined

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,False,False,False,False,False,False,False,False,True,False


In [13]:
combined = combined.drop(columns=['combined nucleotides'])
combined

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,False,False,False,False,False,True,False


In [14]:
combined.value_counts('gene_id')

gene_id
1528    274
977     253
2632    250
2005    230
1929    229
       ... 
857       1
863       1
887       1
2800      1
1412      1
Name: count, Length: 3852, dtype: int64

### Resampling Methods

In [15]:
X = combined.drop('label', axis=1)
y = combined['label']

In [16]:
X

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,False,False,False,False,False,True,False


In [17]:
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek

# ADASYN followed by Tomek Links Pipeline
adasyn_tomek = Pipeline([
    ('adasyn', ADASYN(random_state=42)),
    ('tomek', TomekLinks())
])

# Dictionary of resampling methods
resampling_methods = {
    'RandomUnderSampler': RandomUnderSampler(random_state=42),
    'TomekLinks': TomekLinks(),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42),
    'ADASYN_Tomek': adasyn_tomek
}

### XGBoost with GroupShuffleSplit

In [26]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model = xgb.XGBClassifier(n_estimators=300, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_no_gene_id = X.drop(columns=['gene_id'])

# Loop through each resampling method
for method_name, resampler in resampling_methods.items():
    print(f"Results for {method_name}:")
    
    roc_aucs = []  # List to store ROC AUC scores for each fold
    
    # Loop through each fold
    for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
        X_train_fold, X_valid_fold = X_no_gene_id.iloc[train_index], X_no_gene_id.iloc[valid_index]
        y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
        # Initialize the scaler
        scaler = MinMaxScaler()
        
        # Fit the scaler on the training data and transform both training and validation data
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
        # Apply the current resampling technique
        X_resampled, y_resampled = resampler.fit_resample(X_train_fold_scaled, y_train_fold)
        
        # Train the model on the resampled data
        xgb_model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = xgb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
        threshold = 0.4
        y_pred_binary = (y_pred >= threshold).astype(int)
        
        # Compute the ROC AUC score for this fold and append to the list
        roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
    # Display results for the current resampling method
    print("ROC AUC scores for each fold:", roc_aucs)
    print("Mean ROC AUC score:", np.mean(roc_aucs))
    print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
    print("-" * 50)


Results for RandomUnderSampler:
ROC AUC scores for each fold: [0.8906834928744976, 0.9025330044261164, 0.8913792961119329, 0.8907848352281983, 0.8908846438822484]
Mean ROC AUC score: 0.8932530545045987
Standard deviation of ROC AUC scores: 0.004646128782186436
--------------------------------------------------
Results for TomekLinks:
ROC AUC scores for each fold: [0.8947092831417606, 0.9056361937687386, 0.8998074033799295, 0.8962331317024385, 0.8949339361028431]
Mean ROC AUC score: 0.898263989619142
Standard deviation of ROC AUC scores: 0.0041129332233798095
--------------------------------------------------
Results for SMOTE:
ROC AUC scores for each fold: [0.8879156070698921, 0.8972632747747272, 0.8877335530685254, 0.8815078505606367, 0.8838169433410119]
Mean ROC AUC score: 0.8876474457629587
Standard deviation of ROC AUC scores: 0.005383509461143215
--------------------------------------------------
Results for ADASYN:
ROC AUC scores for each fold: [0.8859953738233554, 0.895510294344

### Gradient Boosting with GroupShuffleSplit

In [29]:
from sklearn.ensemble import GradientBoostingClassifier

# Model initialization using Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=200, random_state=42, learning_rate=0.1, max_depth=5)

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Drop 'gene_id' from X
X_no_gene_id = X.drop(columns=['gene_id'])

# Loop through each resampling method
for method_name, resampler in resampling_methods.items():
    print(f"Results for {method_name}:")
    
    roc_aucs = []  # List to store ROC AUC scores for each fold
    
    # Loop through each fold
    for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
        X_train_fold, X_valid_fold = X_no_gene_id.iloc[train_index], X_no_gene_id.iloc[valid_index]
        y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
        # Initialize the scaler
        scaler = MinMaxScaler()
        
        # Fit the scaler on the training data and transform both training and validation data
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
        # Apply the current resampling technique
        X_resampled, y_resampled = resampler.fit_resample(X_train_fold_scaled, y_train_fold)
        
        # Train the model on the resampled data
        gb_model.fit(X_resampled, y_resampled)
        
        # Predict on the validation set
        y_pred = gb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
        threshold = 0.4
        y_pred_binary = (y_pred >= threshold).astype(int)
        
        # Compute the ROC AUC score for this fold and append to the list
        roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
    print("ROC AUC scores for each fold:", roc_aucs)
    print("Mean ROC AUC score:", np.mean(roc_aucs))
    print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
    print("-" * 50)


Results for RandomUnderSampler:
ROC AUC scores for each fold: [0.8877166902751392, 0.9030808519527316, 0.8922574770377475, 0.8907830573155072, 0.8920635098650983]
Mean ROC AUC score: 0.8931803172892447
Standard deviation of ROC AUC scores: 0.00520994347582129
--------------------------------------------------
Results for TomekLinks:
ROC AUC scores for each fold: [0.893745875478534, 0.8985649177833748, 0.8968498337352057, 0.8974150459510892, 0.890453931347678]
Mean ROC AUC score: 0.8954059208591764
Standard deviation of ROC AUC scores: 0.0029454602487011634
--------------------------------------------------
Results for SMOTE:
ROC AUC scores for each fold: [0.8868696716851079, 0.890790899027647, 0.8815966992442101, 0.8788857334577624, 0.8802821024110032]
Mean ROC AUC score: 0.8836850211651461
Standard deviation of ROC AUC scores: 0.004463397946006506
--------------------------------------------------
Results for ADASYN:


KeyboardInterrupt: 

### 5-mer only

In [32]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model = xgb.XGBClassifier(n_estimators=300, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_min_features = X.drop(columns=['gene_id', 'transcript_id', 'transcript_position', '1A', '1C', '1G', '1T',
                                '7A', '7C', '7G', '7T'])

    
roc_aucs = []  # List to store ROC AUC scores for each fold
    
# Loop through each fold
for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
    X_train_fold, X_valid_fold = X_min_features.iloc[train_index], X_min_features.iloc[valid_index]
    y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
    # Initialize the scaler
    scaler = MinMaxScaler()
        
    # Fit the scaler on the training data and transform both training and validation data
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
    # Apply the current resampling technique
    X_resampled, y_resampled = TomekLinks().fit_resample(X_train_fold_scaled, y_train_fold)
        
    # Train the model on the resampled data
    xgb_model.fit(X_resampled, y_resampled)
        
    # Predict on the validation set
    y_pred = xgb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
    threshold = 0.4
    y_pred_binary = (y_pred >= threshold).astype(int)
        
    # Compute the ROC AUC score for this fold and append to the list
    roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
# Display results for the current resampling method
print("ROC AUC scores for each fold:", roc_aucs)
print("Mean ROC AUC score:", np.mean(roc_aucs))
print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
print("-" * 50)

ROC AUC scores for each fold: [0.8817122298251977, 0.8986595262110875, 0.8834479006959032, 0.8850225266216694, 0.8837309428909161]
Mean ROC AUC score: 0.8865146252489549
Standard deviation of ROC AUC scores: 0.006163467635259012
--------------------------------------------------


### 5-mer & +1 -1

In [33]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model = xgb.XGBClassifier(n_estimators=300, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_min_features = X.drop(columns=['gene_id', 'transcript_id', 'transcript_position'])

    
roc_aucs = []  # List to store ROC AUC scores for each fold
    
# Loop through each fold
for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
    X_train_fold, X_valid_fold = X_min_features.iloc[train_index], X_min_features.iloc[valid_index]
    y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
    # Initialize the scaler
    scaler = MinMaxScaler()
        
    # Fit the scaler on the training data and transform both training and validation data
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
    # Apply the current resampling technique
    X_resampled, y_resampled = TomekLinks().fit_resample(X_train_fold_scaled, y_train_fold)
        
    # Train the model on the resampled data
    xgb_model.fit(X_resampled, y_resampled)
        
    # Predict on the validation set
    y_pred = xgb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
    threshold = 0.4
    y_pred_binary = (y_pred >= threshold).astype(int)
        
    # Compute the ROC AUC score for this fold and append to the list
    roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
# Display results for the current resampling method
print("ROC AUC scores for each fold:", roc_aucs)
print("Mean ROC AUC score:", np.mean(roc_aucs))
print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
print("-" * 50)

ROC AUC scores for each fold: [0.8968412803588648, 0.9053943150903879, 0.8984643114175997, 0.8974740679737195, 0.8941344365914438]
Mean ROC AUC score: 0.8984616822864032
Standard deviation of ROC AUC scores: 0.0037520006462992187
--------------------------------------------------


### 5-mer & +1 -1 & transcript_position

In [34]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model = xgb.XGBClassifier(n_estimators=300, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_min_features = X.drop(columns=['gene_id', 'transcript_id'])

    
roc_aucs = []  # List to store ROC AUC scores for each fold
    
# Loop through each fold
for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
    X_train_fold, X_valid_fold = X_min_features.iloc[train_index], X_min_features.iloc[valid_index]
    y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]
        
    # Initialize the scaler
    scaler = MinMaxScaler()
        
    # Fit the scaler on the training data and transform both training and validation data
    X_train_fold_scaled = scaler.fit_transform(X_train_fold)
    X_valid_fold_scaled = scaler.transform(X_valid_fold)
        
    # Apply the current resampling technique
    X_resampled, y_resampled = TomekLinks().fit_resample(X_train_fold_scaled, y_train_fold)
        
    # Train the model on the resampled data
    xgb_model.fit(X_resampled, y_resampled)
        
    # Predict on the validation set
    y_pred = xgb_model.predict_proba(X_valid_fold_scaled)[:, 1]
        
    threshold = 0.4
    y_pred_binary = (y_pred >= threshold).astype(int)
        
    # Compute the ROC AUC score for this fold and append to the list
    roc_aucs.append(roc_auc_score(y_valid_fold, y_pred))
    
# Display results for the current resampling method
print("ROC AUC scores for each fold:", roc_aucs)
print("Mean ROC AUC score:", np.mean(roc_aucs))
print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))
print("-" * 50)

ROC AUC scores for each fold: [0.8956453928871472, 0.9047844070682846, 0.9008629746951812, 0.8988256559164398, 0.8947647950307028]
Mean ROC AUC score: 0.8989766451195511
Standard deviation of ROC AUC scores: 0.0036372533171979233
--------------------------------------------------
